In [10]:
from elasticsearch import Elasticsearch

# 数据来源:data/example_book_data.jsonl
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [11]:
# The match_phrase query analyzes the text and creates a phrase query out of the analyzed text.
body = {
    "query": {
        "match_phrase": {
            # 查询内容"规划教材"分词(ik_max_word)为: ['规范', '教材']
            "name": {
                # 索引条件解析:
                # [a, b, c, d, e]
                # [a, b] ok
                #    [b, a] no
                # [a, c] no
                "query": "规划教材",
                "boost": 1
            }
        }
    }
}
'''
GET test/_search
`body`
'''
es.search(index='test', body=body)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 205, 'relation': 'eq'},
  'max_score': 2.6843016,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '364',
    '_score': 2.6843016,
    '_source': {'name': '全国成人高等教育法学规划教材：商法',
     'publish': '高等教育出版社',
     'type': '大学教材',
     'author': [],
     'info': '《商法》是教育部统一组织编写的全国成人高等教育法学规划教材《商法》的第三版。《商法》共分七编，分别为总论、公司法、证券法、破产法、票据法、保险法、海商法。全书立足于我国现行商法制度，以简洁的行文涵括了当前商事立法与商法理论的基本内容，构建了较为完整的商法学体系。与第二版相比，因理论发展、立法修订及编写人员构成上的变化，本书全部内容基本上都是重新编写的，从而保证了其内容上的新颖性。',
     'price': 38}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '17',
    '_score': 2.5955868,
    '_source': {'name': '全国高等学校医学规划教材・急诊医学',
     'publish': '高等教育出版社',
     'type': '大学教材',
     'author': ['罗学宏'],
     'info': '《全国高等学校医学规划教材・急诊医学》为普通高等教育“十一五”国家级规划教材，供临床、基础、预防、护理、检验、口腔、药学、麻醉等专业使用，系为满足急诊专科化发展，加速急诊专业人才培养编写而成。《急诊医学》除讲述急诊相关基础理论、基本知识和基本技能外，突出特色是以病案为引导，有的放矢地介绍疾病的常见急症的诊断、鉴别诊断和急诊处理原则，具

In [12]:
es.search(index='test', body={
    "query": {
        "match_phrase": {
            "name": {
                # 查询内容"十一五计算机"分词(ik_max_word)为: ['十一五', '十一', '五', '计算机', '计算', '算机']
                "query": "十一五计算机",
            }
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [13]:
# "国家教委重点教材"的分词结果为:['国家教委', '国家', '家教', '教委', '重点', '教材']
# "国家教材"的分词结果为:['国家', '家教', '教材']
es.search(index='test', body={
    "query": {
        "match_phrase": {
            "name": {
                # 索引条件解析:
                # [a, b, c, d]
                # [a, c] no

                # [a, c] "slop": 1 yes
                # [a, c] -> [a, ?, c]

                # [a, d] [a, b, c, d] "slop": 1 no

                # [a, d] [a, b, c, d] "slop": 2 yes
                # [a, d] -> [a, ?, d] -> [a, ?, ?, d]
                "query": "国家教材",
                "slop": 2
            }
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 3.8026133,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '623',
    '_score': 3.8026133,
    '_source': {'name': '普通高等教育“九五”国家教委重点教材：非线性动力学',
     'publish': '高等教育出版社',
     'type': '大学教材',
     'author': ['刘秉正', '彭建华'],
     'info': '《非线性动力学》比较全面地叙述了非线性动力学的基础知识和基础理论，特别着重介绍了近四十年来发现与研究的非线性系统的混沌运动和对非线性系统特征的一些重要的刻画、分析及诊断方法。《非线性动力学》重视物理概念的阐述和理论联系实际，同时也注意介绍近十余年来这方面的某些重要进展。估计《非线性动力学》的内容和叙述方式无论对理论工作者还是对这方面有兴趣的实际科研人员都是很有好处的，也颇便于刚开始涉猎本领域的初学者。《非线性动力学》可作为理科高年级学生和研究生非线性动力学或混沌力学课程的教材使用，也可供广大科技工作者参考。',
     'price': 46}}]}}

In [14]:
# "全国成人高等教育法学规划教材：商法"的分词结果为:['全国', '成人', '高等教育', '高等', '教育法', '教育', '法学', '规划', '教材', '商法']
# "教材规划"的分词结果为:['教材', '规划']
es.search(index='test', body={
    "query": {
        "match_phrase": {
            "name": {
                # Transposed terms have a slop of 2.
                # 索引条件解析:
                # [a, b, c, d]
                #    [b, a] no
                #    [b, a] "slop":1 no

                #    [b, a] "slop":2 yes
                #    [b, a] ->[b|a] -> [a, b]
                "query": "教材规划",
                "slop": 2
            }
        }
    }
})

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 205, 'relation': 'eq'},
  'max_score': 1.3506796,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '364',
    '_score': 1.3506796,
    '_source': {'name': '全国成人高等教育法学规划教材：商法',
     'publish': '高等教育出版社',
     'type': '大学教材',
     'author': [],
     'info': '《商法》是教育部统一组织编写的全国成人高等教育法学规划教材《商法》的第三版。《商法》共分七编，分别为总论、公司法、证券法、破产法、票据法、保险法、海商法。全书立足于我国现行商法制度，以简洁的行文涵括了当前商事立法与商法理论的基本内容，构建了较为完整的商法学体系。与第二版相比，因理论发展、立法修订及编写人员构成上的变化，本书全部内容基本上都是重新编写的，从而保证了其内容上的新颖性。',
     'price': 38}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '17',
    '_score': 1.2844114,
    '_source': {'name': '全国高等学校医学规划教材・急诊医学',
     'publish': '高等教育出版社',
     'type': '大学教材',
     'author': ['罗学宏'],
     'info': '《全国高等学校医学规划教材・急诊医学》为普通高等教育“十一五”国家级规划教材，供临床、基础、预防、护理、检验、口腔、药学、麻醉等专业使用，系为满足急诊专科化发展，加速急诊专业人才培养编写而成。《急诊医学》除讲述急诊相关基础理论、基本知识和基本技能外，突出特色是以病案为引导，有的放矢地介绍疾病的常见急症的诊断、鉴别诊断和急诊处理原则，具

In [15]:
# "教育部规划教材：建筑工程预算（第3版）"的分词结果为:['教育部', '教育', '部', '规划', '教材', '建筑工程', '建筑', '工程预算', '工程', '预算', '第', '3', '版']
# "教材规划预算"的分词结果为:['教材', '规划', '预算']
'''
解析:
[a, b, c, d, e, f, g]
   [b, a, g]
移动5步即可,解析如下
   [b, a, ?, g] "slop":1
   [b, a, ?, ?, g]  "slop":2
   [b, a, ?, ?, ?, g]   "slop":3
   [b|a, ?, ?, ?, g]   "slop":4
   [a, b, ?, ?, ?, g]   "slop":5
'''
es.search(index='test', body={
    "query": {
        "match_phrase": {
            "name": {
                "query": "教材规划预算",
                "slop": 5
            }
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 2.3988552,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '463',
    '_score': 2.3988552,
    '_source': {'name': '教育部规划教材：建筑工程预算（第3版）',
     'publish': '高等教育出版社',
     'type': '中职中专教材',
     'author': [],
     'info': '《建筑工程预算（第3版）》是教育部职业教育与成人教育司组织编写和修订的全国中等职业学校建筑施工专业系列教材之一，是教育部规划教材。《建筑工程预算（第3版）》以现行的《全国统一建筑工程基础定额》、建筑面积计算规范、工程量清单计价规范、建筑安装工程费用构成等文件为依据编写和修订。与第二版相比，增加了平法图钢筋工程量计算方法和工程量清单计价、人工单价、机械台班单价等内容；改写了工程结算和材料单价及建筑面积计算、建筑安装工程费用等内容。',
     'price': 34}}]}}